In [1]:
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
import mlflow
import mlflow.catboost
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
from catboost import CatBoostRegressor
import numpy as np
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
parent_dir = Path.cwd().parents[1]
sys.path.append(os.path.abspath(parent_dir))

In [4]:
from utils.DataPreprocessPipeline import DataPreprocessPipeline
preprocessor = DataPreprocessPipeline(num_lags=3, rolling_window_size=3)

In [5]:
train_df = pd.read_csv('../../../data/training_data.csv')

In [6]:
train_df.head()

,date_id,item_dept,item_qty,net_sales,store,item,invoice_num
0,11/1/2021,Grocery,1.0,160.0,XYZ,16620,1475459.0
1,11/1/2021,Grocery,2.0,480.0,XYZ,32365,1475459.0
2,11/1/2021,Grocery,1.0,127.0,XYZ,31349,1475459.0
3,11/1/2021,Household,2.0,110.0,XYZ,1266,1475475.0
4,11/1/2021,Household,1.0,150.0,XYZ,114920,1475475.0


In [7]:
train_df_preprocessed = preprocessor.transform(train_df)
train_df_preprocessed

,date_id,item_dept,store,item_qty,net_sales,lag_item_qty_1,lag_item_qty_2,lag_item_qty_3,rolling_mean_item_qty_3,rolling_std_item_qty_3,...,expanding_min_net_sales,expanding_max_net_sales,diff_net_sales,diff_net_sales_7,day_of_week,isWeekend,Is_Holiday,store_XYZ,item_dept_Grocery,item_dept_Household
42,2021-11-08,Beverages,ABC,974.000,246984.68200,937.000,884.000,991.000,937.333333,53.500779,...,199145.596010,2.779048e+05,-23294.307000,57.000,0,0,0,0.0,0.0,0.0
43,2021-11-08,Beverages,XYZ,677.000,201386.08100,1043.000,1086.000,910.000,1013.000000,91.755109,...,187931.769020,2.899930e+05,27566.920000,-102.000,0,0,0,1.0,0.0,0.0
44,2021-11-08,Grocery,ABC,2326.638,438841.43801,2646.820,2614.006,2654.582,2638.469333,21.538411,...,397946.965024,5.822066e+05,-43148.157030,-33.761,0,0,0,0.0,1.0,0.0
45,2021-11-08,Grocery,XYZ,2879.414,498037.79100,3608.962,3625.864,3287.828,3507.551333,190.473559,...,491450.723000,6.616550e+05,38798.438006,-107.646,0,0,0,1.0,1.0,0.0
46,2021-11-08,Household,ABC,883.000,240294.89201,998.000,1152.000,986.000,1045.333333,92.570694,...,209100.513996,2.920205e+05,-50544.119984,-160.000,0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,2022-01-31,Beverages,XYZ,791.000,179122.36501,1048.000,1255.000,1336.000,1213.000000,148.522726,...,149946.090000,5.130570e+05,-97559.236000,22.000,0,0,0,1.0,0.0,0.0
548,2022-01-31,Grocery,ABC,2603.874,542643.93100,2869.215,2587.436,2358.758,2605.136333,255.688411,...,367440.183000,2.035445e+06,33708.435960,286.094,0,0,0,0.0,1.0,0.0
549,2022-01-31,Grocery,XYZ,2996.909,563043.73400,4040.207,3539.002,3162.792,3580.667000,440.188880,...,443513.481000,1.161787e+06,89212.836000,434.602,0,0,0,1.0,1.0,0.0
550,2022-01-31,Household,ABC,1115.000,314800.26999,1170.000,1236.000,1062.000,1156.000000,87.840765,...,181968.999000,4.518945e+05,-26337.569910,180.000,0,0,0,0.0,0.0,1.0


In [8]:
from utils.model_helpers import create_training_testing
train_dict, valid_dict = create_training_testing(train_df_preprocessed, test_date_start='2022-01-15')

In [9]:
train_X = train_dict['train_features']
train_y_sales = train_dict['train_net_sales']
train_y_item_qty = train_dict['train_item_qty']

val_X = valid_dict['train_features']
val_y_sales = valid_dict['train_net_sales']
val_y_item_qty = valid_dict['train_item_qty']

In [10]:
from utils.SalesItemQtyModel import SalesItemQtyModel

In [11]:
def objective(params):
    model_sales = CatBoostRegressor(**params, verbose = 0)
    model_item_qty = CatBoostRegressor(**params, verbose = 0)

    sales_item_qty_model = SalesItemQtyModel(model_sales = model_sales, model_item_qty=model_item_qty)
    sales_item_qty_model.fit(train_X, train_y_sales, train_y_item_qty)

    score_val_sales = sales_item_qty_model.score_sales(val_X, val_y_sales)
    score_val_item_qty = sales_item_qty_model.score_item_qty(val_X, val_y_item_qty)

    with mlflow.start_run(nested=True):
        mlflow.log_params(params)
        mlflow.log_metric("val_sales_MAPE", score_val_sales)
        mlflow.log_metric("val_item_qty_MAPE", score_val_item_qty)
        mlflow.catboost.log_model(model_sales, "model_sales")
        mlflow.catboost.log_model(model_item_qty, "model_item_qty")

    # Objective is to minimize the average of the two MSEs
    return {'loss': (score_val_sales + score_val_item_qty) / 2, 'status': STATUS_OK}

In [12]:
# Define the search space for Hyperopt
search_space = {
    'depth': hp.quniform('depth', 4, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'iterations': scope.int(hp.quniform('iterations', 100, 1000, 50)),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1e-5, 100),
    'border_count': scope.int(hp.quniform('border_count', 32, 255, 1)),
    'bagging_temperature': hp.uniform('bagging_temperature', 0, 1),
    'random_strength': hp.uniform('random_strength', 1e-9, 10),
    'one_hot_max_size': scope.int(hp.quniform('one_hot_max_size', 2, 255, 1)),
}

In [13]:
# Create a Trials object to store the results of the optimization process
trials = Trials()

# Start an MLflow experiment
with mlflow.start_run(run_name="catboost_hyperopt_tuning"):
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=100,
        trials=trials
    )

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

2024/08/31 09:09:14 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2024/08/31 09:09:17 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  1%|          | 1/100 [00:11<18:39, 11.31s/trial, best loss: 0.1509964447667711]

2024/08/31 09:09:21 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2024/08/31 09:09:23 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



  2%|▏         | 2/100 [00:18<14:03,  8.61s/trial, best loss: 0.1509964447667711]

...

 98%|█████████▊| 98/100 [12:10<00:15,  7.67s/trial, best loss: 0.11478333007811639]

2024/08/31 09:21:19 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2024/08/31 09:21:22 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



 99%|█████████▉| 99/100 [12:16<00:07,  7.06s/trial, best loss: 0.11478333007811639]

2024/08/31 09:21:25 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2024/08/31 09:21:28 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



100%|██████████| 100/100 [12:22<00:00,  7.42s/trial, best loss: 0.11478333007811639]


In [14]:
mlflow.end_run()

In [15]:
print("Best hyperparameters found:", best_params)


Best hyperparameters found: {'bagging_temperature': 0.5447824858171504, 'border_count': 220.0, 'depth': 5.0, 'iterations': 800.0, 'l2_leaf_reg': 13.06135266464145, 'learning_rate': 0.15538685598864, 'one_hot_max_size': 34.0, 'random_strength': 4.115171860221108}


In [16]:
best_params

{'bagging_temperature': 0.5447824858171504,
 'border_count': 220.0,
 'depth': 5.0,
 'iterations': 800.0,
 'l2_leaf_reg': 13.06135266464145,
 'learning_rate': 0.15538685598864,
 'one_hot_max_size': 34.0,
 'random_strength': 4.115171860221108}

In [16]:
trials.best_trial

{'state': 2,
 'tid': 39,
 'spec': None,
 'result': {'loss': 0.1169679988465582, 'status': 'ok'},
 'misc': {'tid': 39,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'bagging_temperature': [39],
   'border_count': [39],
   'depth': [39],
   'iterations': [39],
   'l2_leaf_reg': [39],
   'learning_rate': [39],
   'one_hot_max_size': [39],
   'random_strength': [39]},
  'vals': {'bagging_temperature': [0.4772142988105587],
   'border_count': [200.0],
   'depth': [5.0],
   'iterations': [450.0],
   'l2_leaf_reg': [7.43439511907274],
   'learning_rate': [0.22947940228640154],
   'one_hot_max_size': [124.0],
   'random_strength': [5.316275052028734]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2024, 8, 28, 13, 55, 4, 123000),
 'refresh_time': datetime.datetime(2024, 8, 28, 13, 55, 9, 857000)}

In [17]:
CatBoostRegressor(**best_params, verbose = 0)